# Hugging Face Large Model Inference with DistilGPT2
This notebook demonstrates how to deploy the DistilGPT2 transformer model using the HuggingFace Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

TGI is an open source, high performance inference library that can be used to deploy large language models from HuggingFace's repository in minutes. The library includes advanced functionality like model parallelism and dynamic batching to simplify production inference with large language models like flan-t5-xxl, LLaMa, StableLM, and GPT-NeoX.

DistilGPT2 (short for Distilled-GPT2) is an English-language model pretrained with the supervision of the smallest version of Generative Pre-trained Transformer 2 (GPT-2). Similar to GPT-2, DistilGPT2 can be used to generate text.
For more information, please visit the model card on HuggingFace [here](https://huggingface.co/distilgpt2).

## Setup

### Install or update the SageMaker Python SDK
First, we need to make sure the latest version of the SageMaker Python SDK is installed.

In [2]:
!pip install "sagemaker>=2.163.0"


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


### Setup account and role
Then, we import the SageMaker Python SDK and instantiate a `sagemaker_session` which is used to determine the region and execution role.

In [3]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
from sagemaker.utils import name_from_base
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

## Retrieve the LLM Image URI
We use the helper function `get_huggingface_llm_image_uri()` to generate the appropriate image URI for the HuggingFace Large Language Model (LLM) inference.

The function takes a required parameter `backend` and several optional parameters. The `backend` specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI backend, and "huggingface" refers to using HuggingFace TGI backend.

In [4]:
image_uri = get_huggingface_llm_image_uri(backend="huggingface", region=region)

## Create the HuggingFace Model
Next we configure the `model` object by specifying a unique name, the `image_uri` for the managed TGI container, and the execution role for the endpoint. Additionally we specify some environment variables incling the `HF_MODEL_ID` which corresponds to the model in the HuggingFace Hib, and the `HF_TASK` which configures the inference task to be performed.

You should also define `SM_NUM_GPUS` which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here you should set `SM_NUM_GPUS` to the number of available GPUs on your selected instance type. In this example, we will set it to 1 because the selected instance type, `ml.g5.2xlarge` has 1 available GPU.

Additionally, setting `HF_MODEL_QUANTIZE` to `true` will reduce the memory footprint of the model.

In [5]:
hub = {
    "HF_MODEL_ID": "distilgpt2",
    "HF_TASK": "text-generation",
    "SM_NUM_GPUS": "1",
    "HF_MODEL_QUANTIZE": "gptq",
}

model = HuggingFaceModel(name=name_from_base("distilgpt2"), env=hub, role=role, image_uri=image_uri)

## Creating a SageMaker Endpoint
Next we deploy the model using the `deploy()` function. Here we use an `ml.g5.2xlarge` instance which comes with a NVIDIA A10G GPU. By setting the `SM_NUM_GPUS` environment variable to 1, we indicate that the model should only require a single GPU.

In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
)

---------!

## Running Inference
New that the endpoint is running, we can evaluate it using the `predict()` function.

In [7]:
input_data = {
    "inputs": "The diamondback terrapin was the first reptile to",
    "parameters": {"do_sample": True, "max_new_tokens": 100, "temperature": 0.7, "watermark": True},
}

predictor.predict(input_data)

[{'generated_text': 'The diamondback terrapin was the first reptile to successfully live in the United States. This is the third in a line of hybrid hatchlings to be built with the same height of 5 ft. (6 in).\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]

## Cleaning Up
After you have finished using the endpoint, it is important to delete it to avoid incurring unnecessary costs

In [8]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion
In this tutorial, we used a TGI container to deploy DistilGPT2 using a GPU on a SageMaker `ml.g5.2xlarge` instance. With HuggingFace's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl and LLaMa.